<a href="https://colab.research.google.com/github/Chahethsen12/Smart_Support_API/blob/main/Smart_Support_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Industrial Libraries

In [ ]:
!pip install -q fastapi uvicorn pyngrok nest_asyncio transformers torch
print("FastAPI & AI Environment Ready!")

The Full Stack Application

In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import pipeline
from typing import List, Optional
import sqlite3
from datetime import datetime

# 1. THE AI ENGINE (HuggingFace)
# We use a lightweight model to classify text without needing an API key
print("⏳ Loading AI Models... (This takes 30 seconds)")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
print("✅ AI Models Loaded!")

# 2. DATA MODELS (Pydantic - The Industrial Standard)
class TicketCreate(BaseModel):
    customer_name: str
    email: str
    issue_description: str

class TicketResponse(BaseModel):
    id: int
    category: str
    priority: str
    sentiment: str
    status: str

# 3. DATABASE SETUP (SQLite)
conn = sqlite3.connect('support_tickets.db', check_same_thread=False)
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS tickets
             (id INTEGER PRIMARY KEY, customer_name TEXT, issue_description TEXT,
              category TEXT, priority TEXT, sentiment TEXT, status TEXT)''')
conn.commit()

# 4. THE API APP
app = FastAPI(title="SmartHelp AI API", description="Automated Ticket Triage System")

@app.post("/tickets/", response_model=TicketResponse)
async def create_ticket(ticket: TicketCreate):
    # --- INDUSTRIAL LOGIC START ---

    # A. AI Analysis
    # 1. Classify the topic
    labels = ["billing", "technical support", "feature request", "spam"]
    classification = classifier(ticket.issue_description, labels)
    category = classification['labels'][0]

    # 2. Analyze Sentiment (Is the customer angry?)
    sentiment_result = sentiment_analyzer(ticket.issue_description)[0]
    sentiment_score = sentiment_result['label'] # POSITIVE or NEGATIVE

    # B. Business Logic (Rule-based)
    priority = "NORMAL"
    if category == "billing" or sentiment_score == "NEGATIVE":
        priority = "URGENT"

    # C. Save to Database
    c.execute("INSERT INTO tickets (customer_name, issue_description, category, priority, sentiment, status) VALUES (?, ?, ?, ?, ?, ?)",
              (ticket.customer_name, ticket.issue_description, category, priority, sentiment_score, "OPEN"))
    conn.commit()
    ticket_id = c.lastrowid

    # --- INDUSTRIAL LOGIC END ---

    return {
        "id": ticket_id,
        "category": category,
        "priority": priority,
        "sentiment": sentiment_score,
        "status": "OPEN"
    }

@app.get("/tickets/")
async def get_tickets():
    c.execute("SELECT * FROM tickets")
    rows = c.fetchall()
    return [{"id": r[0], "customer": r[1], "issue": r[2], "category": r[3], "priority": r[4], "sentiment": r[5]} for r in rows]

# Allow Colab to run this async
nest_asyncio.apply()

Run the Server

In [ ]:
from threading import Thread

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start server in a background thread
thread = Thread(target=run_server)
thread.start()

print("🚀 Server is running in the background!")
print("Since we are in Colab, we can't visit localhost directly.")
print("Run the NEXT CELL to test your API.")

Test Your "Full Stack" Logic

In [ ]:
import requests
import json

# 1. Simulate a User submitting a ticket
url = "http://127.0.0.1:8000/tickets/"

payload = {
    "customer_name": "John Doe",
    "email": "john@example.com",
    "issue_description": "My credit card was charged twice! I am very angry about this service."
}

print("📤 Sending Ticket to API...")
response = requests.post(url, json=payload)

print("\n📥 API Response (AI Analysis):")
print(json.dumps(response.json(), indent=2))

# Check the results:
# Did the AI catch that John is angry?
# Did it mark it as Billing?
# Did it mark it as URGENT?